In [1]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# need this to ensure reproducibility
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


In [4]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    # inconsistent with keras official example
    # seems not reasonable
    # perhaps we should use bias=True
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
    #                 stride=stride, padding=1)
                     stride=stride, padding=1, bias=False)


# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                # inconsistent with keras
                # seems not reasonable
                # perhaps we should use kernel_size=1 to perform downsampling
                # nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride),
                conv3x3(self.in_channels, out_channels, stride=stride),
                # inconsistent with keras official example
                # seems reasonable
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

# perhaps we should use [3, 3, 3] to keep consistent with resnet20
# model = ResNet(ResidualBlock, [3, 3, 3]).to(device)
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [5]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

195738

In [6]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

    # Decay learning rate
    if (epoch + 1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.7460
Epoch [1/80], Step [200/500] Loss: 1.4734
Epoch [1/80], Step [300/500] Loss: 1.2250
Epoch [1/80], Step [400/500] Loss: 1.3857
Epoch [1/80], Step [500/500] Loss: 1.2048
Epoch [2/80], Step [100/500] Loss: 1.0163
Epoch [2/80], Step [200/500] Loss: 0.8441
Epoch [2/80], Step [300/500] Loss: 1.1287
Epoch [2/80], Step [400/500] Loss: 1.0802
Epoch [2/80], Step [500/500] Loss: 1.0151
Epoch [3/80], Step [100/500] Loss: 1.1108
Epoch [3/80], Step [200/500] Loss: 0.9835
Epoch [3/80], Step [300/500] Loss: 0.8055
Epoch [3/80], Step [400/500] Loss: 0.8650
Epoch [3/80], Step [500/500] Loss: 0.7670
Epoch [4/80], Step [100/500] Loss: 0.9036
Epoch [4/80], Step [200/500] Loss: 0.7709
Epoch [4/80], Step [300/500] Loss: 0.7313
Epoch [4/80], Step [400/500] Loss: 0.6391
Epoch [4/80], Step [500/500] Loss: 0.9977
Epoch [5/80], Step [100/500] Loss: 0.6616
Epoch [5/80], Step [200/500] Loss: 0.8652
Epoch [5/80], Step [300/500] Loss: 0.8531
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [300/500] Loss: 0.2128
Epoch [39/80], Step [400/500] Loss: 0.1966
Epoch [39/80], Step [500/500] Loss: 0.1920
Epoch [40/80], Step [100/500] Loss: 0.1679
Epoch [40/80], Step [200/500] Loss: 0.1872
Epoch [40/80], Step [300/500] Loss: 0.2033
Epoch [40/80], Step [400/500] Loss: 0.1774
Epoch [40/80], Step [500/500] Loss: 0.2105
Epoch [41/80], Step [100/500] Loss: 0.1249
Epoch [41/80], Step [200/500] Loss: 0.2635
Epoch [41/80], Step [300/500] Loss: 0.1659
Epoch [41/80], Step [400/500] Loss: 0.1946
Epoch [41/80], Step [500/500] Loss: 0.1693
Epoch [42/80], Step [100/500] Loss: 0.1458
Epoch [42/80], Step [200/500] Loss: 0.2342
Epoch [42/80], Step [300/500] Loss: 0.2603
Epoch [42/80], Step [400/500] Loss: 0.2142
Epoch [42/80], Step [500/500] Loss: 0.2648
Epoch [43/80], Step [100/500] Loss: 0.2144
Epoch [43/80], Step [200/500] Loss: 0.1379
Epoch [43/80], Step [300/500] Loss: 0.1378
Epoch [43/80], Step [400/500] Loss: 0.1351
Epoch [43/80], Step [500/500] Loss: 0.1812
Epoch [44/8

Epoch [77/80], Step [400/500] Loss: 0.1420
Epoch [77/80], Step [500/500] Loss: 0.1302
Epoch [78/80], Step [100/500] Loss: 0.1899
Epoch [78/80], Step [200/500] Loss: 0.2056
Epoch [78/80], Step [300/500] Loss: 0.1654
Epoch [78/80], Step [400/500] Loss: 0.2419
Epoch [78/80], Step [500/500] Loss: 0.2003
Epoch [79/80], Step [100/500] Loss: 0.2953
Epoch [79/80], Step [200/500] Loss: 0.1066
Epoch [79/80], Step [300/500] Loss: 0.0825
Epoch [79/80], Step [400/500] Loss: 0.1444
Epoch [79/80], Step [500/500] Loss: 0.1281
Epoch [80/80], Step [100/500] Loss: 0.2379
Epoch [80/80], Step [200/500] Loss: 0.1572
Epoch [80/80], Step [300/500] Loss: 0.1617
Epoch [80/80], Step [400/500] Loss: 0.1009
Epoch [80/80], Step [500/500] Loss: 0.0884


In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.21 %


In [9]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')